We are currently working on creating a synthetic dataset where the gains and T60s will be broadband so that we can train the simplest Diff GFDN architecture with scalar gains instead of filters. The synthetic dataset will have two rooms with two different T60s and a set of amplitudes for each source-receiver positions. While we can generate the T60 randomly for the two rooms, generating the amplitudes randomly for each position is not ideal. To generate amplitudes in a meaningful way, we want to analyse the amplitudes at 1kHz in our existing coupled room dataset.

In [ ]:
import numpy as np
import os
import torch
import pickle
import re
from pathlib import Path
import matplotlib.pyplot as plt
from numpy.typing import NDArray, ArrayLike
from typing import List, Optional
from sklearn.mixture import GaussianMixture
from scipy.interpolate import griddata
from scipy.stats import multivariate_normal


from diff_gfdn.config.config import DiffGFDNConfig
from diff_gfdn.dataloader import RoomDataset, ThreeRoomDataset
from diff_gfdn.utils import db

os.chdir('..')  # This changes the working directory to DiffGFDN

### Helper plotting functions

In [ ]:
def plot_amps_at_receiver_points(room_data:RoomDataset, 
                                 rec_pos: NDArray, 
                                 amps: NDArray, 
                                 scatter_plot: bool=True, 
                                 title:Optional[str]=None):
    """
    Plot the amplitudes of the different slopes at specified receiver points.
    Args:
        room_data (RoomDataset): object corresponding to room data
        rec_pos (NDArray): N x 3 array of listener positions in cartesian coordinates
        amps (NDArray): N x num_rooms matrix of amplitudes at specified listener positions
        scatter_plot (bool): whether to plot the discrete amplitudes, 
                             or interpolate them to be continuous functions of space
    """
    x_rec = rec_pos[:, 0]
    y_rec = rec_pos[:, 1]
    source_pos = np.squeeze(room_data.source_position)
    
    fig, ax = plt.subplots(room_data.num_rooms, 1, figsize=(6, 12))
    fig.tight_layout()

    if not scatter_plot:
        # Create a grid for the surface
        num_samps = 1000
        x_lin = np.linspace(0, room_data.room_dims[-1][0] + room_data.room_start_coord[-1][0], num_samps)
        y_lin = np.linspace(0, room_data.room_dims[-1][1] + room_data.room_start_coord[-1][1], num_samps)
        x_mesh, y_mesh = np.meshgrid(x_lin, y_lin)
        
        # Create a mask for values within the limits (so that outside the boundaries the amps are zero)
        mask = []
        combined_mask = np.array([])
        for i in range(room_data.num_rooms):
            cur_mask = (x_mesh >= room_data.room_start_coord[i][0]) & (x_mesh <= room_data.room_dims[i][0] + room_data.room_start_coord[i][0]) & \
                   (y_mesh >= room_data.room_start_coord[i][1]) & (y_mesh <= room_data.room_dims[i][1] + room_data.room_start_coord[i][1])
            if combined_mask.size == 0:
                combined_mask = cur_mask
            else:
                combined_mask = np.logical_or(combined_mask, cur_mask)            

    # Plot the X, Y, Z points
    for i in range(room_data.num_rooms):
        if scatter_plot:
            im = ax[i].scatter(x_rec, y_rec, c=db(amps[i,:], is_squared=True))
            # Set the limits for all axes
            ax[i].set_xlim(0,
                        room_data.room_dims[-1][0] + room_data.room_start_coord[-1][0] + 0.5)
            ax[i].set_ylim(0,
                        room_data.room_dims[-1][1] + room_data.room_start_coord[-1][1] + 0.5)
        else:
            amps_interp = griddata((x_rec, y_rec), amps[i,:], (x_mesh, y_mesh), method='cubic')  # Interpolate z value
            # Set values outside the limits to 0
            amps_interp[~combined_mask] = 0  # Apply the mask
            im = ax[i].imshow(db(amps_interp, is_squared=True), 
                              extent = (0,  room_data.room_dims[-1][0] + room_data.room_start_coord[-1][0],
                                        0,  room_data.room_dims[-1][1] + room_data.room_start_coord[-1][1]),
                              origin='lower', cmap='viridis')        
        fig.colorbar(im, ax=ax[i],orientation='vertical')
    
        # Labels and title
        ax[i].scatter(source_pos[0],
                      source_pos[1],
                      color='red',
                      marker='x',
                      s=50)
        ax[i].set_xlabel('X axis')
        ax[i].set_ylabel('Y axis')
        ax[i].set_title(f'1kHz amplitudes for slope = {i+1}')

    # Show the plot
    if title is not None:
        plt.suptitle(title)
    fig.subplots_adjust(hspace=0.2)
    # fig.colorbar(im, ax=ax, orientation='vertical', fraction=0.05, pad=0.05)

    plt.show()

# Function to plot ellipsoids
def plot_ellipsoid(mean : ArrayLike, cov_matrix: NDArray, ax, color='r', alpha=0.2):
    """Plot the GMM parameters as ellipsoids on a scatter plot"""
    # Decompose the covariance matrix to get the principal axes and radii
    U, s, Vt = np.linalg.svd(cov_matrix)
    radii = np.sqrt(s)
    
    # Generate data for ellipsoid surface
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    x = radii[0] * np.outer(np.cos(u), np.sin(v))
    y = radii[1] * np.outer(np.sin(u), np.sin(v))
    z = radii[2] * np.outer(np.ones_like(u), np.cos(v))

    # Rotate the ellipsoid data with the eigenvectors
    for i in range(len(x)):
        for j in range(len(x)):
            [x[i, j], y[i, j], z[i, j]] = np.dot([x[i, j], y[i, j], z[i, j]], U) + mean
    
    # Plot the ellipsoid
    ax.plot_surface(x, y, z, color=color, alpha=alpha)

### Read data and plot amplitudes as a function of space

In [ ]:
config_dict = DiffGFDNConfig()
print(Path(config_dict.room_dataset_path).resolve())
room_data = ThreeRoomDataset(Path(config_dict.room_dataset_path).resolve(), config_dict)
config_dict = config_dict.copy(update={"num_groups": room_data.num_rooms})

common_decay_times = room_data.common_decay_times
band_centre_hz = np.array(room_data.band_centre_hz)
amplitudes = np.array(room_data.amplitudes)
num_rec_pos = room_data.num_rec

In [ ]:
# get amplitudes at 1kHz
mid_band_freq = 1000.0
idx = np.argwhere(band_centre_hz == mid_band_freq)[0][0]
amplitudes_mid_band = amplitudes[idx, ...]

plot_amps_at_receiver_points(room_data, np.array(room_data.receiver_position), amplitudes_mid_band, scatter_plot=False)

### Stats helper functions

In [ ]:
def fit_multivariate_normal_params(data: NDArray):
    """Get the mean and covariance matrix of a multivariate gaussian"""
    n, d = data.shape
    # get mean vector of size d x 1
    mean = np.mean(data, axis=0)
    covariance = ((data-mean).T @ (data-mean)) / (n-1)
    return mean, covariance

def sample_gmm(means: List, covariances : List, pis: List, component_choices: ArrayLike, n_samples:int):
    """
    Sample from a GMM with means, covariances and mixing coefficient pi
    Args:
        means (List): means of size num_mixtures x num_features
        covariances (List): covariance matrix of size num_mixtures x num_features x num_features
        pis (List): weight of each mixture (Weights must sum to 1).
        component_choise (List): which normal distribution in the mixture to sample from, of size n_samples
        n_samples (int): number of samples to return
    Returns:
        NDArray: sampled daata from GMM of shape n_samples x num_features
    """
    K = len(pis)  # Number of components
    d = means.shape[1]  # Dimensionality of data
    
    # Step 2: Sample from the chosen Gaussian component
    samples = np.zeros((n_samples, d))
    
    for i in range(n_samples):
        k = component_choices[i]  # Chosen component
        # Sample from the corresponding multivariate normal
        samples[i, :] = np.random.multivariate_normal(means[k], covariances[k])
    
    return samples


### Fit the amplitudes to a distribution

#### Fitting to Gaussian Mixture Model

If our data is multimodal, and the amplitudes are correlated, then we can model it as a linear mixture of $K$ gaussians,
\begin{aligned}
p({A_{k, \mathbf{x}}}) &= \frac{\exp \left[-(A_{k, \mathbf{x}} - \mathbf{\mu_k})^T \mathbf{\Sigma_k}^{-1} (A_{k, \mathbf{x}} - \mathbf{\mu_k}) \right]}{\sqrt{(2 \pi)^k \text{det} (\mathbf{\Sigma}_k)}} \\
p(A_{\mathbf{x}}) &= \sum_{k=1}^K \pi_k p({A_{k, \mathbf{x}}})
\end{aligned}

where $A_{k, \mathbf{x}}$ is the amplitude corresponding to the $k$th group at position $\mathbf{x}$. It is derived from a gaussian mixture model with $K$ mixtures (one corresponding to each common decay time). We wish to find the parameters $\mathbf{\Sigma_k}$ and $\mathbf{\mu_k}$ of this distribution, and then sample $A_{k,\mathbf{x}}$ from it. To do this, we will use the expectation maximisation algorithm.

#### Fitting to a single multivariate Gaussian
We can also assume a single normal distribution for the amplitudes corresponding to each slope, and calculate the mean and covariance of those. Here, we assume that the amplitudes corresponding to room 1, are completely independent of those in rooms 2 and 3, i.e, $A_{1, \mathbf{x}} \perp A_{2, \mathbf{x}} \perp  \ldots A_{K, \mathbf{x}}$

To check which of these is a valid choice, let's see if the data is unimodal or multimodal (should be modeled with GMM). We also fit a GMM to the data and checked the covariance matrices which had significantly large off-diagonal elements, confirming that the amplitudes in the three rooms are not independent.

**From the scatter plot, the GMM is the right choice for modeling this data. The GMM ellipsoids are shown in green and match the 3 distinct clusters. The multivariate Gaussian ellipsoid is shown in red and does not match the 3 clusters.**

In [ ]:
# mean is of shape num_groups x num_feature
# weight is of size num_groups
# covariance is of size num_groups x num_feature x num_feature
# the groups are not ordered, but we can explicitly order them. The group with the largest mean
# is likely going to belong to room 1 (as it contains the source), the second largest to room 2 and 
# third largest to room 3 (see plots above).

def sort_gmm_by_means(gmm):
    """Sort GMM components by the means along the first feature dimension."""
    
    # Sort in descending order by average value of the means
    sorted_indices = np.flip(np.argsort(np.mean(gmm.means_, axis=-1)))
    
    # Reorder the GMM parameters based on the sorted indices
    gmm.means_ = gmm.means_[sorted_indices]
    gmm.covariances_ = gmm.covariances_[sorted_indices]
    gmm.weights_ = gmm.weights_[sorted_indices]
    
    # If GMM has precisions (used in certain covariance types), reorder them as well
    if hasattr(gmm, 'precisions_'):
        gmm.precisions_ = gmm.precisions_[sorted_indices]
    if hasattr(gmm, 'precisions_cholesky_'):
        gmm.precisions_cholesky_ = gmm.precisions_cholesky_[sorted_indices]
    
    # Reorder the labels (optional, if relevant)
    gmm.predict_labels_ = sorted_indices
    return gmm


# get parameters of GMM, data of shape (N, 3)
data = amplitudes_mid_band.T
gmm = GaussianMixture(n_components=3)
gmm.fit(data)
sort_gmm_by_means(gmm)

# save to a pickle file
gmm_dict = {'means': gmm.means_, "covariance": gmm.covariances_, "weights": gmm.weights_}
with open(Path('resources/Georg_3room_FDTD/amplitude_distribution.pkl').resolve(), 'wb') as f:
    pickle.dump(gmm_dict, f)
print("Mean vector for all K groups:\n", db(gmm.means_))
print("Weigths for all K groups:\n", db(gmm.weights_))


# get parameters of a multivariate normal distribution
mu_n, sigma_n = fit_multivariate_normal_params(amplitudes_mid_band.T)
# print the covariance matrix to see if it is a diagonal
print("Covariance matrix (should be diagonal for independent variables):\n", sigma_n)


# Create a 3D scatter plot
# Plot the scatter plot
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data[:, 0], data[:, 1], data[:, 2], s=10, c='blue', marker='o', label='Data')

# Plot ellipsoids for each covariance matrix
for i in range(room_data.num_rooms):
    plot_ellipsoid(gmm.means_[i], gmm.covariances_[i], ax, color='green')
plot_ellipsoid(mu_n, sigma_n, ax, color='red') 

# Labels and title
ax.set_xlabel('Room 1')
ax.set_ylabel('Room 2')
ax.set_zlabel('Room 3')
plt.title('GMM Covariance Matrices and Data')
plt.show()

The question we are yet to answer is - how do we make the amplitudes a function of the room geometry and source-receiver positions? Here are some empirical rules we follow to design this function.

1. The strength of the amplitudes should be inversely proportional to the distance between the source and the receiver.
2. If the receiver is in a particular room, then the amplitudes of the slopes corresponding
   to that room should dominate.

We propose the following function,
$$
A(k, \mathbf{x}) = \frac{1}{\sqrt{d(\text{src, rec}) * d(\text{rec, room}_k) }} * \pi_k \mu_{k,\text{rec-room}}
$$
where $d(.)$ is the Euclidean distance between 2 points, $d(\text{rec, room}_k)$ is the distance between the receiver and the $k$th room, and $\mu_{k,\text{rec-room}}$ is the $k$th term of the mean vector corresponding to the room where the receiver is located.

### Geometry helper functions

In [ ]:
def generate_random_positions(N: int, L: ArrayLike, U: ArrayLike):
    """
    Generate N x 3 matrix of uniformly distributed cartesian coordinates, where the value in each column
    is constrained by its own range (L_k, U_k).

    Parameters:
    - N (int): Number of rows (samples)
    - L (list or array): Lower bounds for each column (size 3)
    - U (list or array): Upper bounds for each column (size 3)
    
    Returns:
    - np.ndarray: N x 3 array of random numbers
    """
    assert len(L) == len(U) == 3, "L and U must both have same size"
    
    # Generate N random values for each column in the range (L_k, U_k)
    x = np.random.uniform(L[0], U[0], size=N)
    y = np.random.uniform(L[1], U[1], size=N)
    z = np.random.uniform(L[2], U[2], size=N)
    
    # Stack columns together to form N x 3 array
    return np.column_stack((x, y, z))

def get_euclidean_distance(point1: NDArray, point2: NDArray, ax:int):
    """Get the euclidean distance between a set of points"""
    return np.sqrt(np.sum((point1 - point2)**2, axis=ax))

def is_point_in_room(corners: List, point :ArrayLike):
    """
    Check if a point is inside a quadrilateral.
    
    Parameters:
    - corners (list of tuples): List of 4 (x, y) coordinates for the quadrilateral.
    - point (tuple): The (x, y) coordinate of the point to check.
    
    Returns:
    - bool: True if the point is inside the quadrilateral, False otherwise.
    """
    def cross_product(A, B, P):
        # Compute the 2D cross product of vectors AB and AP
        return (B[0] - A[0]) * (P[1] - A[1]) - (B[1] - A[1]) * (P[0] - A[0])
    
    # Extract the four corners and the point
    A, B, C, D = corners
    P = point
    
    # Compute the cross products for all four edges
    cross1 = cross_product(A, B, P)
    cross2 = cross_product(B, C, P)
    cross3 = cross_product(C, D, P)
    cross4 = cross_product(D, A, P)
    
    # Check if all cross products have the same sign
    if (cross1 > 0 and cross2 > 0 and cross3 > 0 and cross4 > 0) or \
       (cross1 < 0 and cross2 < 0 and cross3 < 0 and cross4 < 0):
        return True
    else:
        return False

def point_to_room_distance(P: ArrayLike, room_bounds: List):
    """
    Get the distance from a point to a room edge, if the point
    is inside the room, the distance is 1e-3. Works only in 2D for now.
    Args:
        P (ArrayLike): 2D point
        room_bounds (List): [min_x, max_x, min_y, max_y] corresponding to 2D room bounds
    """
    Px, Py = P
    x_min, x_max, y_min, y_max = room_bounds
    
    # Distance in the x direction
    dx = max(x_min - Px, 0, Px - x_max)
    
    # Distance in the y direction
    dy = max(y_min - Py, 0, Py - y_max)
    
    # Overall distance (Euclidean distance)
    dist = np.sqrt(dx**2 + dy**2)
    return 1e-3 if dist <= 1e-3 else dist
    
        
def get_amplitude_based_on_position(room_start_coord :List, room_dims : List, rec_pos : NDArray, source_pos: ArrayLike, 
                                    mean_amp: ArrayLike):
    """
    Get amplitudes corresponding to K common slopes, given the room geometry and the mean amplitude
    """
    num_rooms = len(room_start_coord)
    num_rec = rec_pos.shape[0]
    rec_pos = np.stack((rec_pos[:, 0], rec_pos[:, 1]), axis=0)
    amplitudes = np.zeros((num_rooms, num_rec))
    
    # get the boundaries for all the rooms
    room_boundaries = [[[room_start_coord[i][0], room_start_coord[i][1]], 
                       [room_start_coord[i][0] + room_dims[i][0], room_start_coord[i][1]],
                       [room_start_coord[i][0] + room_dims[i][0], room_start_coord[i][1] + room_dims[i][1]],
                       [room_start_coord[i][0], room_start_coord[i][1] + room_dims[i][1]]] for i in range(num_rooms)]

    room_midpoint = [(np.array([room_start_coord[i][0], room_start_coord[i][1]]) + \
                     np.array([room_start_coord[i][0] + room_dims[i][0], room_start_coord[i][1] + room_dims[i][1]]))/2.0 \
                     for i in range(num_rooms)]
    
    dist_from_room = np.zeros((num_rooms, num_rec))
    dist_from_source = get_euclidean_distance(np.repeat(source_pos[:2, np.newaxis], num_samples, axis=1), rec_pos, ax=0)
    point_in_room = np.empty((num_rooms, num_rec), dtype=bool)
    for k in range(num_rooms):
        # find out which room the receiver is in
        point_in_room[k, :] = np.array([is_point_in_room(room_boundaries[k], rec_pos[:, i]) for i in range(num_rec)])
        
        # distance of the receivers from the room
        dist_from_room[k,:] = np.array([point_to_room_distance(rec_pos[:, i], \
                                                               [room_start_coord[k][0], room_start_coord[k][0] + room_dims[k][0],
                                                                room_start_coord[k][1], room_start_coord[k][1] + room_dims[k][1]]) \
                                        for i in range(num_rec)])
        
        # amplitudes depend on 1/r
        amplitudes[k, :] = 1.0 / (np.sqrt(dist_from_room[k, :] * dist_from_source) + 1e-12)

    # scale the amplitudes by the mean of the amplitudes
    for j in range(num_rec):
        which_room = np.argwhere(point_in_room[:, j])[0][0]
        amplitudes[:,j] *= mean_amp[which_room]
        

    return amplitudes

In [ ]:
# number of receiver positions
num_samples = 100
mean_amps = np.ones((room_data.num_rooms, room_data.num_rooms))
mean_amps = gmm.weights_ * gmm.means_

# create listening points in room 1
for k in range(room_data.num_rooms):
    lower_bound = np.array(room_data.room_start_coord[k])
    upper_bound = np.array(room_data.room_start_coord[k]) + np.array(room_data.room_dims[k])
    sampled_positions = generate_random_positions(num_samples, lower_bound, upper_bound)
    amplitudes_sampled = get_amplitude_based_on_position(room_data.room_start_coord, room_data.room_dims, 
                                                         sampled_positions, np.squeeze(room_data.source_position),
                                                         mean_amps)
    plot_amps_at_receiver_points(room_data, 
                                 sampled_positions, 
                                 amplitudes_sampled, scatter_plot=False, title=f'Receivers in room {k+1}')
    # break